# Spaceship Titanic

Our goal is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly.


## File and Data Field Descriptions

* **train.csv** - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.
    * PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
    * HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
    * CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
    * Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
    * Destination - The planet the passenger will be debarking to.
    * Age - The age of the passenger.
    * VIP - Whether the passenger has paid for special VIP service during the voyage.
    * RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
    * Name - The first and last names of the passenger.
    * Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.
* **test.csv** - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of Transported for the passengers in this set.
* **sample_submission.csv** - A submission file in the correct format.
    * PassengerId - Id for each passenger in the test set.
    * Transported - The target. For each passenger, predict either True or False.

## Libraries imports

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 1) Dataset loading and preprocess

First of all, we load the training and test sets.

In [2]:
# Load a dataset into a Pandas Dataframe
# Try to load the dataset from Kaggle, if not found, load from local directory
try:
    train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
    test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
except FileNotFoundError:
    train_df = pd.read_csv('kaggle/input/spaceship-titanic/train.csv')
    test_df = pd.read_csv('kaggle/input/spaceship-titanic/test.csv')

print("Full train dataset shape is {}".format(train_df.shape))
print("Full test dataset shape is {}".format(test_df.shape))

Full train dataset shape is (8693, 14)
Full test dataset shape is (4277, 13)


In [3]:
# I split the datasets into features (X) and tag (Y)
train_x = train_df.drop(columns=['Transported'])
train_y = train_df['Transported'].astype(int)  # Convert boolean to int (0 or 1)

test_x = test_df

To evaluate the different models used, I split the training set into train and validation, giving a 15% of the samples to the validation set. I set that percentage in order to have a dataset big enough to evaluate the models.

In [4]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.15, random_state=0)

print("Train dataset shape is {}".format(train_x.shape))
print("Validation dataset shape is {}".format(val_x.shape))
train_x.head()

Train dataset shape is (7389, 13)
Validation dataset shape is (1304, 13)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
4549,4841_01,Europa,True,B/192/S,55 Cancri e,59.0,False,0.0,0.0,0.0,0.0,0.0,Mahamak Prefule
1182,1254_02,Mars,False,F/241/S,TRAPPIST-1e,27.0,False,4.0,0.0,896.0,229.0,0.0,Babix Breke
4064,4339_01,Mars,False,F/814/S,TRAPPIST-1e,47.0,False,2044.0,0.0,95.0,0.0,120.0,Ranch Reste
6156,6493_01,Earth,True,G/1050/P,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Terald Dillips
5548,5915_01,Earth,False,E/382/S,TRAPPIST-1e,32.0,False,0.0,0.0,39.0,0.0,749.0,Garion Mcneiley


After separate a validation set from our training set, the next step is to preprocess the dataset, to do that I will check each individual feature.

### PassengerId

This variable represents a unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.

At the begining it seems to be a variant without information to solve the problem, something like a name, however as it said, we can extract the number of members that each group has and that can be something relevant.
However to know the number of members a group has, we have to use the whole dataset, trining + validation + test sets. To do this would be a problem in most cases because we are not sure that our dataset contain all the existent samples, however for this task, the description tells us that we have about two-thirds (~8700) of the passengers as the training set and THE REMAINING one-third (~4300) of the passengers as test, so we got at least partial information about all the passengers.

In [5]:
# Check for missing values in PassengerId
print("Missing values in PassengerId:")
print(f"train_x: {train_x['PassengerId'].isnull().sum()}")
print(f"test_x: {test_x['PassengerId'].isnull().sum()}")
print(f"val_x: {val_x['PassengerId'].isnull().sum()}")

# Extract group identifier (gggg) from PassengerId
train_x['Group'] = train_x['PassengerId'].str.split('_').str[0]
test_x['Group'] = test_x['PassengerId'].str.split('_').str[0]
val_x['Group'] = val_x['PassengerId'].str.split('_').str[0]

# Combine all dataframes to calculate group sizes across all datasets
combined_df = pd.concat([train_x, val_x, test_x])

# Calculate the total number of members in each group
group_sizes = combined_df['Group'].value_counts()

# Add the numMembers column to each dataframe
train_x['numMembers'] = train_x['Group'].map(group_sizes)
val_x['numMembers'] = val_x['Group'].map(group_sizes)
test_x['numMembers'] = test_x['Group'].map(group_sizes)

# Display the updated dataframes
print("Updated train_x:")
print(train_x[['PassengerId', 'Group', 'numMembers']].head())

print("\nUpdated val_x:")
print(val_x[['PassengerId', 'Group', 'numMembers']].head())

print("\nUpdated test_x:")
print(test_x[['PassengerId', 'Group', 'numMembers']].head())


Missing values in PassengerId:
train_x: 0
test_x: 0
val_x: 0
Updated train_x:
     PassengerId Group  numMembers
4549     4841_01  4841           3
1182     1254_02  1254           2
4064     4339_01  4339           1
6156     6493_01  6493           1
5548     5915_01  5915           1

Updated val_x:
     PassengerId Group  numMembers
3601     3868_05  3868           7
6057     6405_02  6405           4
2797     3021_01  3021           2
7110     7578_01  7578           1
8579     9158_01  9158           1

Updated test_x:
  PassengerId Group  numMembers
0     0013_01  0013           1
1     0018_01  0018           1
2     0019_01  0019           1
3     0021_01  0021           1
4     0023_01  0023           1


Now we can remove the columns PassengerId and Group, that don't give us any more information, however I will wait and remove all the useless columns after analyze all the features.

### HomePlanet 
The planet the passenger departed from, typically their planet of permanent residence.

This is a categorical feature, so we will need to encode it in some way in order to give the information to the models.

In [6]:
# First I check how many different values we have in this variant (HomePlanet)
print("HomePlanet unique values: ", train_x['HomePlanet'].unique())

# And also if there are any missing values and how many
print("Missing values in HomePlanet: ", train_x['HomePlanet'].isnull().sum(), f"({train_x['HomePlanet'].isnull().sum()/len(train_x)*100:.2f}%)")

HomePlanet unique values:  ['Europa' 'Mars' 'Earth' nan]
Missing values in HomePlanet:  167 (2.26%)


First of all, let's solve the issue with the missing values, we can assume that members of the same group depart from the same planet, so the first approach will be to assign the existing value of one member of the group to other members that have a null value in the HomePlanet variable.

In [7]:
# Check if all members of the same group have the same HomePlanet
group_destination_check = train_x.groupby('Group')['HomePlanet'].nunique()

# Find groups with more than one unique HomePlanet
inconsistent_groups = group_destination_check[group_destination_check > 1]

# Print the result
if inconsistent_groups.empty:
    print("All members of the same group have the same HomePlanet.")
else:
    print("Some groups have inconsistent HomePlanet:")
    print(inconsistent_groups)

All members of the same group have the same HomePlanet.


In [8]:
combined_df = pd.concat([train_x, val_x])

# Fill missing HomePlanet values based on the group
combined_df['HomePlanet'] = combined_df.groupby('Group')['HomePlanet'].transform(lambda x: x.ffill().bfill())
test_x['HomePlanet'] = test_x.groupby('Group')['HomePlanet'].transform(lambda x: x.ffill().bfill())

# Update train_x, val_x, and test_x with the filled values from combined_df based on PassengerId
train_x['HomePlanet'] = train_x['PassengerId'].map(combined_df.set_index('PassengerId')['HomePlanet'])
val_x['HomePlanet'] = val_x['PassengerId'].map(combined_df.set_index('PassengerId')['HomePlanet'])

# Verify if there are still missing values in HomePlanet
print("Missing values in HomePlanet after filling:")
print(f"train_x: {train_x['HomePlanet'].isnull().sum()}")
print(f"val_x: {val_x['HomePlanet'].isnull().sum()}")
print(f"test_x: {test_x['HomePlanet'].isnull().sum()}")

C:\Users\javie\AppData\Local\Temp\ipykernel_24364\165029983.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['HomePlanet'] = combined_df.groupby('Group')['HomePlanet'].transform(lambda x: x.ffill().bfill())
C:\Users\javie\AppData\Local\Temp\ipykernel_24364\165029983.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_x['HomePlanet'] = test_x.groupby('Group')['HomePlanet'].transform(lambda x: x.ffill().bfill())


Missing values in HomePlanet after filling:
train_x: 92
val_x: 19
test_x: 46


In [9]:
# Check an example to see that everything is correct, in this case the group 0064 come from Mars
combined_df[combined_df['Name'] == 'Colatz Keen']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Group,numMembers
59,0064_02,Mars,True,E/3/S,TRAPPIST-1e,33.0,False,0.0,0.0,NaN,0.0,0.0,Colatz Keen,0064,2


We filled ~50% of the empty values based on the training set, however we still have a little bit more than a 1% of empty entries, so now I will check the class distribution, maybe there is one popular class and I can simply give that class value to the empty entries.

In [10]:
# Class distribution for the HomePlanet column
homeplanet_distribution = train_x['HomePlanet'].value_counts(dropna=False)
homeplanet_percentages = (train_x['HomePlanet'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("HomePlanet distribution (counts):")
print(homeplanet_distribution)
print("\nHomePlanet distribution (percentages):")
print(homeplanet_percentages)

HomePlanet distribution (counts):
HomePlanet
Earth     3904
Europa    1848
Mars      1545
NaN         92
Name: count, dtype: int64

HomePlanet distribution (percentages):
HomePlanet
Earth     52.84
Europa    25.01
Mars      20.91
NaN        1.25
Name: proportion, dtype: float64


Seeing the distribution, if I assign Earth to the empty values, I will aim ~53% of the cases, taking into account that we have 1.25% of empty values, I will have ~0.6% of incorrect values with this approach, this is a very low percentage, so I go with this plan.

In [11]:
# Fill NaN values in the HomePlanet column with 'Earth'
train_x['HomePlanet'] = train_x['HomePlanet'].fillna('Earth')
val_x['HomePlanet'] = val_x['HomePlanet'].fillna('Earth')
test_x['HomePlanet'] = test_x['HomePlanet'].fillna('Earth')

# Verify if there are still missing values in HomePlanet
print("Missing values in HomePlanet after filling with 'Earth':")
print(f"train_x: {train_x['HomePlanet'].isnull().sum()}")
print(f"val_x: {val_x['HomePlanet'].isnull().sum()}")
print(f"test_x: {test_x['HomePlanet'].isnull().sum()}")

Missing values in HomePlanet after filling with 'Earth':
train_x: 0
val_x: 0
test_x: 0


Finally, in order to train the model with this feature, we need to encode it, to do that I will use one-hot encoding that will add 1 more variable to the problem.

The idea is to pass from HomePlanet to two boolean variables, isHomeEarth and isHomeEuropa, if both variants are False we still have the info that the HomePlanet is Mars without having to explicitly saving into another variant. With this approach I am assuming that the only possible HomePlanets are Earth, Mars and Europa, which is not crazy to say seeing the class distribution in the training set. 

In [12]:
# One-hot encoding for HomePlanet
train_x['isHomeEarth'] = (train_x['HomePlanet'] == 'Earth').astype(int)
train_x['isHomeEuropa'] = (train_x['HomePlanet'] == 'Europa').astype(int)

val_x['isHomeEarth'] = (val_x['HomePlanet'] == 'Earth').astype(int)
val_x['isHomeEuropa'] = (val_x['HomePlanet'] == 'Europa').astype(int)

test_x['isHomeEarth'] = (test_x['HomePlanet'] == 'Earth').astype(int)
test_x['isHomeEuropa'] = (test_x['HomePlanet'] == 'Europa').astype(int)

# Verify the new columns
train_x[['HomePlanet', 'isHomeEarth', 'isHomeEuropa']].head()

,HomePlanet,isHomeEarth,isHomeEuropa
4549,Europa,0,1
1182,Mars,0,0
4064,Mars,0,0
6156,Earth,1,0
5548,Earth,1,0


### CryoSleep 
Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.

This is a boolean variable, so we only have to cast it into integer, however first I need to check if there are any empty value and the class distribution.

In [13]:
# Print the distribution of CryoSleep
cryosleep_distribution = train_x['CryoSleep'].value_counts(dropna=False)
cryosleep_percentages = (train_x['CryoSleep'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("CryoSleep distribution (counts):")
print(cryosleep_distribution)
print("\nCryoSleep distribution (percentages):")
print(cryosleep_percentages)

print("Missing values in CryoSleep: ", train_x['CryoSleep'].isnull().sum(), f"({train_x['CryoSleep'].isnull().sum()/len(train_x)*100:.2f}%)")

CryoSleep distribution (counts):
CryoSleep
False    4619
True     2585
NaN       185
Name: count, dtype: int64

CryoSleep distribution (percentages):
CryoSleep
False    62.51
True     34.98
NaN       2.50
Name: proportion, dtype: float64
Missing values in CryoSleep:  185 (2.50%)


Similar than with the HomePlanet, we have ~2.5% of empty values.

In this case, we have the statement where if a passenger is in cryosleep, he is confined inside his cabin, so he won't spend any money on services, that means that variables RoomService, FoodCourt, ShoppingMall, Spa and VRDeck will be all 0.

To confirm this, lets check:
* When CryoSleep == True, sum(RoomService, FoodCourt, ShoppingMall, Spa and VRDeck) == 0.0
* When CryoSleep == False, sum(RoomService, FoodCourt, ShoppingMall, Spa and VRDeck) > 0.0

To do this check I will exclude samples with any of this fields empty.

In [14]:
# Exclude samples with any of the relevant fields empty
filtered_df = train_x.dropna(subset=['CryoSleep', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'])

# Check the conditions
cryosleep_true = filtered_df[filtered_df['CryoSleep'] == True]
cryosleep_false = filtered_df[filtered_df['CryoSleep'] == False]

cryosleep_true_condition = cryosleep_true[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1).eq(0.0)
cryosleep_false_condition = cryosleep_false[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1).gt(0.0)

# Print the results
print("CryoSleep == True and all services == 0.0:")
print(cryosleep_true_condition.all())
print(f"Number of samples: {len(cryosleep_true)}")
if not cryosleep_true_condition.all():
    print("\nRows where CryoSleep == True and all services != 0.0:")
    print(cryosleep_true[~cryosleep_true_condition])

print("\nCryoSleep == False and sum of services > 0.0:")
print(cryosleep_false_condition.all())
print(f"Number of samples: {len(cryosleep_false)}")
if not cryosleep_false_condition.all():
    print("\nRows where CryoSleep == False and sum of services == 0.0:")
    print(cryosleep_false[~cryosleep_false_condition])

CryoSleep == True and all services == 0.0:
True
Number of samples: 2282

CryoSleep == False and sum of services > 0.0:
False
Number of samples: 4161

Rows where CryoSleep == False and sum of services == 0.0:
     PassengerId HomePlanet CryoSleep     Cabin  Destination   Age    VIP  \
3626     3903_01      Earth     False   G/636/P  55 Cancri e   2.0  False   
4015     4280_01       Mars     False   E/267/P  TRAPPIST-1e  24.0  False   
2002     2137_03       Mars     False   F/423/P  55 Cancri e   4.0  False   
3733     4001_07      Earth     False   G/659/S  TRAPPIST-1e   3.0    NaN   
7529     8052_01      Earth     False  G/1302/S  TRAPPIST-1e   9.0  False   
...          ...        ...       ...       ...          ...   ...    ...   
4305     4592_01       Mars     False   F/862/S  TRAPPIST-1e   9.0  False   
7316     7830_04      Earth     False  G/1268/S  TRAPPIST-1e   0.0  False   
3455     3715_03      Earth     False   G/602/P  TRAPPIST-1e   2.0  False   
2418     2595_01      

Well, with the obtained results we can assume that if a passenger has spent something, he is not on cryosleep, however we CANNOT assume that if a passenger didn't spend anything, he is on cryosleep.

So let's do a recap, taking into account the distribution (~62.5% False -- ~35% True), the most probable is that inside that ~2.5% of empty values this distribution also apply, so let's apply the rule where if a passenger has spent something, he is not on cryosleep and see the new distribution to evaluate the next steps.

In [15]:
# When 'CryoSleep' is nan, if a passenger has spent something on 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa' or 'VRDeck', he is not on cryosleep, so we can fill 'CryoSleep' as False
train_x.loc[train_x['CryoSleep'].isnull() & (train_x[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) > 0.0), 'CryoSleep'] = False
val_x.loc[val_x['CryoSleep'].isnull() & (val_x[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) > 0.0), 'CryoSleep'] = False
test_x.loc[train_x['CryoSleep'].isnull() & (test_x[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) > 0.0), 'CryoSleep'] = False

# Print the updated distribution of CryoSleep
cryosleep_distribution = train_x['CryoSleep'].value_counts(dropna=False)
cryosleep_percentages = (train_x['CryoSleep'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("CryoSleep distribution (counts):")
print(cryosleep_distribution)
print("\CryoSleep distribution (percentages):")
print(cryosleep_percentages)

print("Missing values in CryoSleep: ", train_x['CryoSleep'].isnull().sum(), f"({train_x['CryoSleep'].isnull().sum()/len(train_x)*100:.2f}%)")

CryoSleep distribution (counts):
CryoSleep
False    4721
True     2585
NaN        83
Name: count, dtype: int64
\CryoSleep distribution (percentages):
CryoSleep
False    63.89
True     34.98
NaN       1.12
Name: proportion, dtype: float64
Missing values in CryoSleep:  83 (1.12%)


We had 185 NaN values and now we have only 83, meaning that from that set of 185 samples, the ~55% of the samples are not CryoSleep and the remain percentage is still unknown, taking into account that approx. the ~62% of the samples inside a set are not CryoSleep, there is still a ~7% of the samples that can not be CryoSleep, this means that there are ~13 samples that are not CryoSleep, but didn't spend a penny. In total percentages this means that from the 7389 samples that the training set has, a ~0.18% of them will have this field wrong. So we can take the risk of assign CryoSleep == True to the rest of NaN values.

In [16]:
# Fill NaN values in the CryoSleep column with True
train_x['CryoSleep'] = train_x['CryoSleep'].fillna(True)
val_x['CryoSleep'] = val_x['CryoSleep'].fillna(True)
test_x['CryoSleep'] = test_x['CryoSleep'].fillna(True)

# Verify if there are still missing values in CryoSleep
print("Missing values in CryoSleep after filling with True:")
print(f"train_x: {train_x['CryoSleep'].isnull().sum()}")
print(f"val_x: {val_x['CryoSleep'].isnull().sum()}")
print(f"test_x: {test_x['CryoSleep'].isnull().sum()}")

Missing values in CryoSleep after filling with True:
train_x: 0
val_x: 0
test_x: 0


C:\Users\javie\AppData\Local\Temp\ipykernel_24364\1584031860.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_x['CryoSleep'] = train_x['CryoSleep'].fillna(True)
C:\Users\javie\AppData\Local\Temp\ipykernel_24364\1584031860.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_x['CryoSleep'] = val_x['CryoSleep'].fillna(True)
C:\Users\javie\AppData\Local\Temp\ipykernel_24364\1584031860.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=F

Finally, let's cast the boolean values into integers.

In [17]:
train_x['CryoSleep'] = train_x['CryoSleep'].astype(int)
val_x['CryoSleep'] = val_x['CryoSleep'].astype(int)
test_x['CryoSleep'] = test_x['CryoSleep'].astype(int)

train_x.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Group,numMembers,isHomeEarth,isHomeEuropa
4549,4841_01,Europa,1,B/192/S,55 Cancri e,59.0,False,0.0,0.0,0.0,0.0,0.0,Mahamak Prefule,4841,3,0,1
1182,1254_02,Mars,0,F/241/S,TRAPPIST-1e,27.0,False,4.0,0.0,896.0,229.0,0.0,Babix Breke,1254,2,0,0
4064,4339_01,Mars,0,F/814/S,TRAPPIST-1e,47.0,False,2044.0,0.0,95.0,0.0,120.0,Ranch Reste,4339,1,0,0
6156,6493_01,Earth,1,G/1050/P,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Terald Dillips,6493,1,1,0
5548,5915_01,Earth,0,E/382/S,TRAPPIST-1e,32.0,False,0.0,0.0,39.0,0.0,749.0,Garion Mcneiley,5915,1,1,0


### Cabin
The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard. So first let's separate each value and save them in different columns, but even before that, let's see if multiple passengers can stare in the same cabin.

In [18]:
# Number of unique values in Cabin
print("Number of unique values in Cabin:", train_x['Cabin'].nunique()) # Not counting NaN values
print("Number of total values in Cabin:", len(train_x['Cabin']))

# Print the Cabin value with the most occurrences and its count
most_common_cabin = train_x['Cabin'].mode()[0]
print("Most common Cabin value:", most_common_cabin)
print("Count of most common Cabin value:", train_x['Cabin'].value_counts().max())

Number of unique values in Cabin: 5719
Number of total values in Cabin: 7389
Most common Cabin value: B/11/S
Count of most common Cabin value: 7


That means that several passenger can be found in the same cabin. Let's now see the number of missing values, because maybe we can restore them based on the Group of the passenger.

In [19]:
# Print the number of missing values in Cabin
print("Missing values in Cabin:", train_x['Cabin'].isnull().sum(), f"({train_x['Cabin'].isnull().sum()/len(train_x)*100:.2f}%)")

Missing values in Cabin: 158 (2.14%)


In [20]:
# Check if all members of the same Group have the same Cabin
group_destination_check = train_x.groupby('Group')['Cabin'].nunique()

# Find groups with more than one unique Cabin
inconsistent_groups = group_destination_check[group_destination_check > 1]

# Print the result
if inconsistent_groups.empty:
    print("All members of the same group have the same Cabin.")
else:
    print("Some groups have inconsistent Cabin:")
    print(inconsistent_groups)

Some groups have inconsistent Cabin:
Group
0006    2
0064    2
0067    2
0091    2
0099    2
       ..
9128    2
9208    2
9219    2
9220    2
9231    2
Name: Cabin, Length: 331, dtype: int64


So it's not secured that members of the same group travel in the same cabin. So let's now split the Cabin string into their values.

In [21]:
# Split the Cabin column into deck, num, and side
train_x[['CabinDeck', 'CabinNum', 'CabinSide']] = train_x['Cabin'].str.split('/', expand=True)
val_x[['CabinDeck', 'CabinNum', 'CabinSide']] = val_x['Cabin'].str.split('/', expand=True)
test_x[['CabinDeck', 'CabinNum', 'CabinSide']] = test_x['Cabin'].str.split('/', expand=True)

# Convert CabinNum to numeric for potential numerical analysis
train_x['CabinNum'] = pd.to_numeric(train_x['CabinNum'], errors='coerce').astype('Int64')
val_x['CabinNum'] = pd.to_numeric(val_x['CabinNum'], errors='coerce').astype('Int64')
test_x['CabinNum'] = pd.to_numeric(test_x['CabinNum'], errors='coerce').astype('Int64')

# Display the updated train_x dataframe
train_x[['Cabin', 'CabinDeck', 'CabinNum', 'CabinSide']].head()

,Cabin,CabinDeck,CabinNum,CabinSide
4549,B/192/S,B,192,S
1182,F/241/S,F,241,S
4064,F/814/S,F,814,S
6156,G/1050/P,G,1050,P
5548,E/382/S,E,382,S


With the data divided in deck/num/side, let's now see the distribution and the nan values.

In [22]:
# Check the distribution and missing values for CabinDeck, CabinNum, and CabinSide
for column in ['CabinDeck', 'CabinNum', 'CabinSide']:
    print(f"Distribution for {column}:")
    print(train_x[column].value_counts(dropna=False))
    print(f"\nMissing values in {column}: {train_x[column].isnull().sum()} ({train_x[column].isnull().mean() * 100:.2f}%)\n")

Distribution for CabinDeck:
CabinDeck
F      2374
G      2154
E       768
B       669
C       629
D       409
A       223
NaN     158
T         5
Name: count, dtype: int64

Missing values in CabinDeck: 158 (2.14%)

Distribution for CabinNum:
CabinNum
<NA>    158
82       25
19       20
230      19
65       19
       ... 
1870      1
1532      1
1812      1
1761      1
744       1
Name: count, Length: 1764, dtype: Int64

Missing values in CabinNum: 158 (2.14%)

Distribution for CabinSide:
CabinSide
S      3653
P      3578
NaN     158
Name: count, dtype: int64

Missing values in CabinSide: 158 (2.14%)



In [23]:
# Distribution of CabinDeck based on CabinSide
for side in ['S', 'P']:
    print(f"Distribution for CabinSide = {side}:\n")
    
    # CabinDeck distribution
    cabin_deck_distribution = train_x[train_x['CabinSide'] == side]['CabinDeck'].value_counts(dropna=False)
    print(f"CabinDeck distribution:\n{cabin_deck_distribution}\n")

Distribution for CabinSide = S:

CabinDeck distribution:
CabinDeck
F    1143
G    1093
E     379
B     372
C     344
D     200
A     121
T       1
Name: count, dtype: int64

Distribution for CabinSide = P:

CabinDeck distribution:
CabinDeck
F    1231
G    1061
E     389
B     297
C     285
D     209
A     102
T       4
Name: count, dtype: int64



In [24]:
# Distribution of CabinNum based on CryoSleep
for side in [True, False]:
    print(f"Distribution for CryoSleep = {side}:\n")
    
    # CabinDeck distribution
    cabin_deck_distribution = train_x[train_x['CryoSleep'] == side]['CabinDeck'].value_counts(dropna=False)
    cabin_deck_percentages = (cabin_deck_distribution / cabin_deck_distribution.sum() * 100).round(2)
    
    # Combine counts and percentages into a DataFrame
    cabin_deck_distribution_df = pd.DataFrame({
        'Count': cabin_deck_distribution,
        'Percentage': cabin_deck_percentages
    })
    
    # Calculate percentage relative to total passengers in each CabinDeck
    total_cabin_deck_counts = train_x['CabinDeck'].value_counts(dropna=False)
    cabin_deck_distribution_df['Percentage of Total CabinDeck'] = (
        cabin_deck_distribution / total_cabin_deck_counts * 100
    ).round(2)
    
    print(cabin_deck_distribution_df)

Distribution for CryoSleep = True:

           Count  Percentage  Percentage of Total CabinDeck
CabinDeck                                                  
G           1185       44.42                          55.01
F            478       17.92                          20.13
B            371       13.91                          55.46
C            261        9.78                          41.49
E            157        5.88                          20.44
D             87        3.26                          21.27
NaN           67        2.51                          42.41
A             62        2.32                          27.80
Distribution for CryoSleep = False:

           Count  Percentage  Percentage of Total CabinDeck
CabinDeck                                                  
F           1896       40.16                          79.87
G            969       20.53                          44.99
E            611       12.94                          79.56
C            368        7.7

In [25]:
# Check if the probability of being transported is biased on CabinSide
aux_df = train_x.copy()
aux_df['Transported'] = train_y

cabin_side_distribution = aux_df.groupby('CabinSide')['Transported'].value_counts(normalize=True).unstack()

print("\nProbability of being transported based on CabinSide:")
print(cabin_side_distribution)


Probability of being transported based on CabinSide:
Transported         0         1
CabinSide                      
P            0.548351  0.451649
S            0.443197  0.556803


In [26]:
# Check if the probability of being transported is biased on CabinDeck
aux_df = train_x.copy()
aux_df['Transported'] = train_y

cabin_side_distribution = aux_df.groupby('CabinDeck')['Transported'].value_counts(normalize=True).unstack()

print("\nProbability of being transported based on CabinDeck:")
print(cabin_side_distribution)


Probability of being transported based on CabinDeck:
Transported         0         1
CabinDeck                      
A            0.502242  0.497758
B            0.255605  0.744395
C            0.325914  0.674086
D            0.574572  0.425428
E            0.636719  0.363281
F            0.563184  0.436816
G            0.477252  0.522748
T            0.800000  0.200000


Seeing the results we can observe that the same decks exists on both sides, so we are not obtaining any information from that. As I see it, the CabinDeck and CabinNum don't give us any useful information and are categorical variants with lots of different values, so in order to not increase exponentially the complexity of the problem, I will remove them. On the other side, the CabinSide consist in only two different values that we can encode into a single boolean variant, however, it seems that it also doesn't give us any useful or discriminatory information, so we will remove all these features.

### Destination 
The planet the passenger will be debarking to.

Similar to the HomePlanet, this is a categorical value, so let's again see the class distribution and the number of unique values that exists.

In [27]:
# First I check how many different values we have in this variant (Destination)
print("Destination unique values: ", train_x['Destination'].unique())

# And also if there are any missing values and how many
print("Missing values in Destination: ", train_x['Destination'].isnull().sum(), f"({train_x['Destination'].isnull().sum()/len(train_x)*100:.2f}%)")

Destination unique values:  ['55 Cancri e' 'TRAPPIST-1e' 'PSO J318.5-22' nan]
Missing values in Destination:  158 (2.14%)


First of all, let's solve the issue with the missing values, we can assume that members of the same group have the same destination, so the first approach will be to assign the existing value of one member of the group to other members that have a null value in the Destination variable.

In [28]:
# Check if all members of the same group have the same Destination
group_destination_check = train_x.groupby('Group')['Destination'].nunique()

# Find groups with more than one unique Destination
inconsistent_groups = group_destination_check[group_destination_check > 1]

# Print the result
if inconsistent_groups.empty:
    print("All members of the same group have the same Destination.")
else:
    print("Some groups have inconsistent Destinations:")
    print(inconsistent_groups)

Some groups have inconsistent Destinations:
Group
0008    2
0020    3
0044    2
0067    3
0099    2
       ..
9205    2
9219    2
9220    2
9227    2
9280    2
Name: Destination, Length: 559, dtype: int64


Well, we can't follow this approach now, let's see the class distribution in more detail to see the frequency of each value.

In [29]:
# Class distribution for the Destination column
destination_distribution = train_x['Destination'].value_counts(dropna=False)
destination_percentages = (train_x['Destination'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("Destination distribution (counts):")
print(destination_distribution)
print("\nDestination distribution (percentages):")
print(destination_percentages)

Destination distribution (counts):
Destination
TRAPPIST-1e      5039
55 Cancri e      1524
PSO J318.5-22     668
NaN               158
Name: count, dtype: int64

Destination distribution (percentages):
Destination
TRAPPIST-1e      68.20
55 Cancri e      20.63
PSO J318.5-22     9.04
NaN               2.14
Name: proportion, dtype: float64


Most of the passengers go to TRAPPIST-1e, I could simply give that Destination to all the empty values.

In [30]:
# Fill NaN values in Destination with 'TRAPPIST-1e'
train_x['Destination'] = train_x['Destination'].fillna('TRAPPIST-1e')
val_x['Destination'] = val_x['Destination'].fillna('TRAPPIST-1e')
test_x['Destination'] = test_x['Destination'].fillna('TRAPPIST-1e')

Now let's just do the one-hot encoding to pass this variable to two boolean features that tell if destination isTrappist or isCancri.

In [31]:
# One-hot encoding for Destination
train_x['isTrappist'] = (train_x['Destination'] == 'TRAPPIST-1e').astype(int)
train_x['isCancri'] = (train_x['Destination'] == '55 Cancri e').astype(int)

val_x['isTrappist'] = (val_x['Destination'] == 'TRAPPIST-1e').astype(int)
val_x['isCancri'] = (val_x['Destination'] == '55 Cancri e').astype(int)

test_x['isTrappist'] = (test_x['Destination'] == 'TRAPPIST-1e').astype(int)
test_x['isCancri'] = (test_x['Destination'] == '55 Cancri e').astype(int)

# Verify the new columns
train_x[['Destination', 'isTrappist', 'isCancri']].head()

,Destination,isTrappist,isCancri
4549,55 Cancri e,0,1
1182,TRAPPIST-1e,1,0
4064,TRAPPIST-1e,1,0
6156,PSO J318.5-22,0,0
5548,TRAPPIST-1e,1,0


### Age 
The age of the passenger.

This is a numerical variable that doesn't need to apply any encoding, let's see it distribution in detail and if there is any empty value.

In [32]:
# Check the distribution of Age
age_distribution = train_x['Age'].describe()
print("Age distribution:")
print(age_distribution)

# Check for missing values in Age
missing_age_train = train_x['Age'].isnull().sum()

print("\nMissing values in Age:")
print(f"train_x: {missing_age_train} ({missing_age_train / len(train_x) * 100:.2f}%)")

Age distribution:
count    7233.000000
mean       28.822757
std        14.471338
min         0.000000
25%        19.000000
50%        27.000000
75%        38.000000
max        79.000000
Name: Age, dtype: float64

Missing values in Age:
train_x: 156 (2.11%)


Same as the other features, we have a ~2% of empty values, let's use the mean to fill the NaN values.

In [33]:
# Let's use the mean to fill the NaN values.
train_x['Age'] = train_x['Age'].fillna(train_x['Age'].mean())
val_x['Age'] = val_x['Age'].fillna(val_x['Age'].mean())
test_x['Age'] = test_x['Age'].fillna(test_x['Age'].mean())

### VIP 
Whether the passenger has paid for special VIP service during the voyage. This is a boolean feature. 

In [34]:
# Print the distribution of VIP
VIP_distribution = train_x['VIP'].value_counts(dropna=False)
VIP_percentages = (train_x['VIP'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("VIP distribution (counts):")
print(VIP_distribution)
print("\nVIP distribution (percentages):")
print(VIP_percentages)

print("Missing values in VIP: ", train_x['VIP'].isnull().sum(), f"({train_x['VIP'].isnull().sum()/len(train_x)*100:.2f}%)")

VIP distribution (counts):
VIP
False    7041
NaN       181
True      167
Name: count, dtype: int64

VIP distribution (percentages):
VIP
False    95.29
NaN       2.45
True      2.26
Name: proportion, dtype: float64
Missing values in VIP:  181 (2.45%)


In [35]:
# Filter passengers where CryoSleep is True
cryo_sleep_true = train_x[train_x['CryoSleep'] == 1]

# Calculate the VIP distribution for passengers in CryoSleep
vip_distribution_cryo_sleep = cryo_sleep_true['VIP'].value_counts(dropna=False)
vip_percentages_cryo_sleep = (cryo_sleep_true['VIP'].value_counts(normalize=True, dropna=False) * 100).round(2)

# Print the distribution
print("VIP distribution when CryoSleep is True:")
print(vip_distribution_cryo_sleep)
print(vip_percentages_cryo_sleep)

VIP distribution when CryoSleep is True:
VIP
False    2580
NaN        71
True       17
Name: count, dtype: int64
VIP
False    96.70
NaN       2.66
True      0.64
Name: proportion, dtype: float64


Taking into account that in a normal distribution only ~2.29% of the samples have VIP == True, that means that from our set of NaN values, we will only have 4 or 5 wrong values (Passenger with VIP == True where we set VIP == False), so I will set all empty VIP values to False.

In [36]:
# Fill NaN values in the VIP column with False
train_x['VIP'] = train_x['VIP'].fillna(False)
val_x['VIP'] = val_x['VIP'].fillna(False)
test_x['VIP'] = test_x['VIP'].fillna(False)

# Verify if there are still missing values in VIP
print("Missing values in VIP after filling with False:")
print(f"train_x: {train_x['VIP'].isnull().sum()}")
print(f"val_x: {val_x['VIP'].isnull().sum()}")
print(f"test_x: {test_x['VIP'].isnull().sum()}")

Missing values in VIP after filling with False:
train_x: 0
val_x: 0
test_x: 0


C:\Users\javie\AppData\Local\Temp\ipykernel_24364\1786103325.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_x['VIP'] = train_x['VIP'].fillna(False)
C:\Users\javie\AppData\Local\Temp\ipykernel_24364\1786103325.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_x['VIP'] = val_x['VIP'].fillna(False)
C:\Users\javie\AppData\Local\Temp\ipykernel_24364\1786103325.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-

### RoomService, FoodCourt, ShoppingMall, Spa, VRDeck 
Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities. 

These are all numerical features, let's check their distribution and if there are missing values.

In [37]:
# Check the distribution and missing values for RoomService, FoodCourt, ShoppingMall, Spa, and VRDeck
amenities = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

for amenity in amenities:
    print(f"Distribution for {amenity}:")
    print(train_x[amenity].describe())
    print(f"\nMissing values in {amenity}: {train_x[amenity].isnull().sum()} ({train_x[amenity].isnull().mean() * 100:.2f}%)\n")

Distribution for RoomService:
count     7231.000000
mean       230.348776
std        683.342179
min          0.000000
25%          0.000000
50%          0.000000
75%         51.500000
max      14327.000000
Name: RoomService, dtype: float64

Missing values in RoomService: 158 (2.14%)

Distribution for FoodCourt:
count     7232.000000
mean       453.941372
std       1579.658236
min          0.000000
25%          0.000000
50%          0.000000
75%         77.000000
max      27723.000000
Name: FoodCourt, dtype: float64

Missing values in FoodCourt: 157 (2.12%)

Distribution for ShoppingMall:
count     7207.000000
mean       177.340364
std        627.384360
min          0.000000
25%          0.000000
50%          0.000000
75%         28.000000
max      23492.000000
Name: ShoppingMall, dtype: float64

Missing values in ShoppingMall: 182 (2.46%)

Distribution for Spa:
count     7229.000000
mean       308.818232
std       1113.812818
min          0.000000
25%          0.000000
50%          0.0

We know from previous observations that when a passenger is on CryoSleep, he won't spend a penny on amenities, so let's fill with 0 these rows where CryoSleep is True.

In [38]:
# Fill amenities with 0 where CryoSleep is True
for col in amenities:
    train_x.loc[train_x['CryoSleep'] == 1, col] = train_x.loc[train_x['CryoSleep'] == 1, col].fillna(0.0)
    val_x.loc[val_x['CryoSleep'] == 1, col] = val_x.loc[val_x['CryoSleep'] == 1, col].fillna(0.0)
    test_x.loc[test_x['CryoSleep'] == 1, col] = test_x.loc[test_x['CryoSleep'] == 1, col].fillna(0.0)

In [39]:
for amenity in amenities: print(f"Missing values in {amenity}: {train_x[amenity].isnull().sum()} ({train_x[amenity].isnull().mean() * 100:.2f}%)")

Missing values in RoomService: 96 (1.30%)
Missing values in FoodCourt: 96 (1.30%)
Missing values in ShoppingMall: 98 (1.33%)
Missing values in Spa: 105 (1.42%)
Missing values in VRDeck: 93 (1.26%)


Now for the rest of empty values, let's fill them with the mean value for that amenity.

In [40]:
# Fill remaining NaN values in amenities with the mean value for each amenity
for col in amenities:
    train_x[col] = train_x[col].fillna(train_x[col].mean())
    val_x[col] = val_x[col].fillna(val_x[col].mean())
    test_x[col] = test_x[col].fillna(test_x[col].mean())

### Unused columns
After analyze all the features, let's get rid of the useless columns.

In [41]:
train_x.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Group', 'numMembers', 'isHomeEarth', 'isHomeEuropa',
       'CabinDeck', 'CabinNum', 'CabinSide', 'isTrappist', 'isCancri'],
      dtype='object')

In [42]:
columns_to_drop = ['PassengerId', 'HomePlanet', 'Cabin', 'Destination', 'Name', 'Group', 'CabinDeck', 'CabinNum', 'CabinSide']
train_x = train_x.drop(columns=columns_to_drop)
val_x = val_x.drop(columns=columns_to_drop)
test_x = test_x.drop(columns=columns_to_drop)